In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

lst = ['city_destination', 'observation_date', 'delivery_date','observation_date_y','observation_date_w', 
         'observation_date_d','delivery_date_y', 'delivery_date_w', 'delivery_date_d',
         'total_booked_cancelation_prediction','total_cancelations_rs','running_sum_bookings',
         'total_cancelations_rs_log', 'total_booked_cancelation_prediction_log','total_bookings_rs_log',
         'lead_time']
target_column = 'target_demanded'

test_size_ = 0.2
random_state_ = 42

df = pd.read_csv('data/daily_test.csv')
df = df.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)

In [2]:
one_more_city = pd.unique(df.city_destination)

In [3]:
df_append = pd.DataFrame()
df_res = pd.DataFrame()

for i in one_more_city:
    df_init = df[df.city_destination == i]
    
    ml_df = df_init.drop(lst, axis = 1).dropna() 
    
    X = ml_df.drop(columns=target_column) 
    y = ml_df[target_column]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size_, random_state=random_state_)
        
    sc = StandardScaler()
    X_train_ = sc.fit_transform(X_train)
    X_test_ = sc.transform(X_test)
    
    model = LinearRegression()
    model.fit(X_train_, y_train)
    
    df_res['city'] = [i]
    df_res['intercept'] = model.intercept_
    df_res['exp_mov_avg4'] = model.coef_[0]
    df_res['exp_mov_avg5'] = model.coef_[1]
    df_res['exp_mov_avg6'] = model.coef_[2]
    df_res['top_booker_amt'] = model.coef_[3]
    
    df_append = pd.concat([df_res, df_append]).reset_index(drop=True)

In [4]:
df_append

,city,intercept,exp_mov_avg4,exp_mov_avg5,exp_mov_avg6,top_booker_amt
0,Manila,0.000000,0.000000,0.000000,0.000000,0.000000
1,Oslo,5.618235,2.128348,-0.248923,0.123112,0.796088
2,Christchurch,1.012956,0.543023,0.330573,-0.329580,0.268961
3,Melbourne,10.214312,1.712120,0.232625,0.341179,-0.097885
4,Haikou,0.023247,0.000000,0.000000,0.000000,0.051707
...,...,...,...,...,...,...
169,Madrid,37.279211,8.799342,-15.713544,18.466127,1.909805
170,Cape Town,11.238891,3.571007,-1.889808,2.104997,-0.239875
171,Amman,0.660492,0.079257,0.039230,-0.041441,0.135382
172,Bonn,0.594624,0.066732,0.110744,-0.112031,0.242564
